In [1]:
import pandas as pd

from airsenal.framework.utils import *

In [2]:
query = (
    session.query(PlayerScore)
            .filter(PlayerScore.minutes >= 60)
)


In [3]:
df = pd.read_sql(query.statement, engine)
df.head()

,id,player_team,opponent,points,goals,assists,bonus,conceded,minutes,player_id,...,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bps,influence,creativity,threat,ict_index
0,41,WHU,WAT,9,1,0,2,1,90,427,...,0,0,0,0,0,33,44.6,12.5,21.0,7.8
1,42,WHU,NOR,3,0,0,0,0,84,427,...,0,0,0,0,0,20,16.2,18.7,4.0,3.9
2,43,WHU,AVL,2,0,0,0,0,90,427,...,0,0,1,0,0,2,4.2,4.5,31.0,4.0
3,44,WHU,MUN,5,0,1,0,0,90,427,...,0,0,1,0,0,16,4.4,14.8,7.0,2.6
4,45,WHU,BOU,2,0,0,0,2,74,427,...,0,0,0,0,0,7,0.0,2.8,6.0,0.8


In [4]:
keep_player_ids = df.groupby("player_id").bonus.count() >= 10
keep_player_ids = keep_player_ids[keep_player_ids == True].index


In [5]:
avg_bonus = df.groupby("player_id").bonus.mean().sort_values(ascending=False)
avg_bonus.head(20)

player_id
924     2.000000
1042    2.000000
264     1.500000
1127    1.500000
302     1.285714
1024    1.153846
315     1.076923
831     1.017241
171     1.000000
240     1.000000
1047    1.000000
742     1.000000
657     1.000000
10      1.000000
4       0.974026
268     0.909091
366     0.905660
460     0.901408
388     0.900000
256     0.888889
Name: bonus, dtype: float64

In [6]:
get_player("302").name

'Bruno Miguel Borges Fernandes'

In [7]:
get_player("Trent Alexander-Arnold").player_id

259